In this file we calculate the nested core-user. 
- first sampling and then finding 50% core users
- for sampling we use LLS and for finding core-user we use column select from CUR.
- we do all this for the 0-1 matrix, since we need to get non-core-user and feed to NUEMF algorithm.

In [12]:
from fileinput import filename
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import svds
import random 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import scipy.stats as ss
import pickle
from sklearn.utils.extmath import randomized_svd
from scipy.sparse.linalg import svds

In [4]:
def ReadData():
    ml1m_dir = 'data/ratings.dat'
    ml1m_rating = pd.read_csv(ml1m_dir, sep='::', header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')
    unique_uid = np.unique(np.array(ml1m_rating['uid'].tolist()))
    unique_mid = np.unique(np.array(ml1m_rating['mid'].tolist()))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    mid_dict = dict([(y,x) for x,y in enumerate(unique_mid)])
    print('DICTIONARY PREPARED:')

    # init user item dictionary:
    
    uid_list = ml1m_rating['uid'].tolist()
    uid_list_len = len(uid_list)
    mid_list = ml1m_rating['mid'].tolist()
    mid_list_len = len(mid_list)
    rating_list = ml1m_rating['rating'].tolist()
    user_item_dict = {x:set() for x in range(len(unique_uid))}
    item_user_dict = {x:set() for x in range(len(unique_mid))}
    for i in range(uid_list_len):
        uid_list[i] = uid_dict[uid_list[i]]
        mid_list[i] = mid_dict[mid_list[i]]
        rating_list[i] = 1 # comment this line if you want to activate explicit ratings
        user_item_dict[uid_list[i]].add(mid_list[i])
        item_user_dict[mid_list[i]].add(uid_list[i])
    tmp_df = pd.DataFrame({"uid":uid_list, "mid":mid_list, "ratings":rating_list})

    return tmp_df, len(np.unique(mid_list)), len(unique_uid)

In [5]:
# threshold = 30 #split the users into test and train by threshold number of interactions. if greater than threshold then all interactions of that user goes into train set.
df, unique_mids, unique_uids = ReadData()
print("UNIQUE MIDS: ", unique_mids)
print("UNIQUE UIDS: ", unique_uids)

DICTIONARY PREPARED:
UNIQUE MIDS:  3706
UNIQUE UIDS:  6040


In [6]:
core_user_ko_input_train_df = df

- utility functions for CUR coreusers.

In [7]:
def ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)
    
    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])
    
    for index, row in dataframe.iterrows():
        userItemMatrix[row['uid']][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    df = pd.DataFrame(userItemMatrix)
    cosineSimilarity = cosine_similarity(df)
    print("SHAPE OF COSINE MATIX:\n ", cosineSimilarity.shape)

    listToStoreTopFiftyOfEveryUser = []
    for i in range(0, cosineSimilarity.shape[0]):
        idx = np.argpartition(cosineSimilarity[i], -50)[-50:]
        listToStoreTopFiftyOfEveryUser.append(idx)
    # print("Top fifty list: \n", listToStoreTopFiftyOfEveryUser)
    # listToStoreTopFiftyOfEveryUser = np.array(listToStoreTopFiftyOfEveryUser)
    flatten = np.concatenate(listToStoreTopFiftyOfEveryUser)
    listToStoreTopFiftyOfEveryUser = flatten.ravel()

    # print("List of top 50", listToStoreTopFiftyOfEveryUser)
    df = pd.DataFrame(listToStoreTopFiftyOfEveryUser)
    allUserList = df.value_counts().index.tolist()
    # print("ALL USERS LIST", allUserList)
    allUserList = list(sum(allUserList,()))
    # print("ALL USERS LIST", allUserList)
    twentyPercentUserList = allUserList[:int(len(allUserList)*0.2)]
    # print("TWENTY PERCENT USER:", len(twentyPercentUserList))
    # print("TWENTY PERCENT USER:", (twentyPercentUserList))
    
    r_ind = twentyPercentUserList
    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cos_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # coreusers = dataframe.iloc[np.where(dataframe.uid.isin(twentyPercentUserList))]
    # coreusers.reset_index()
    # print("CORE USERS:\n", coreusers)
    return cos_coreusers

In [13]:
MAX_MID = 27277 + 1
def select_cols(mat, k, dup=False):
    # prob 1d array of probabilities of all columns
    prob = mat.T.dot(mat)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis = 0)
    prob = prob/denom

    C = np.zeros((mat.shape[0], k))
    ind_cols = np.arange(0, prob.size)
    c_ind = []
    i = 0
    while(i < k):
        rand_sel = np.random.choice(ind_cols, 1, p=prob)
        if rand_sel in c_ind:
            continue
        c_ind.append(rand_sel[0])
        C[:, i] = mat[:, rand_sel[0]]
        i += 1
        # C[:, i] = C[:, i]/np.sqrt(k*prob[rand_sel[0]])

    return C, c_ind

def select_rows(mat, k, dup=False):

    prob = mat.dot(mat.T)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis=0)
    prob = prob/denom
    print(prob)
    r = np.zeros((k, mat.shape[1]))
    ind_rows = np.arange(0, prob.size)
    r_ind = []
    i = 0
    while(i < k):
        # print(ind_rows)
        rand_sel = np.random.choice(ind_rows, 1, p=prob)
        if rand_sel in r_ind:
            continue
        r_ind.append(rand_sel[0])
        r[i, :] = mat[rand_sel[0], :]
        i += 1
        # r[i, :] = r[i, :]/np.sqrt(k*prob[rand_sel[0]])
    r_ind = np.array(r_ind)
    return r, r_ind

In [14]:
def CUR_ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[row['uid']]][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    mat = userItemMatrix
    print("MAT:", mat)
    print(mat.shape)
    C, c_ind = select_cols(mat, int(u_len * percent_core_user)) ## getting 20% core users
    r, r_ind= select_rows(mat, int(u_len * percent_core_user))
    # print("r", r)
    # print("r_ind len", len(r_ind))

    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

In [16]:
class largest_leveragescores_Sampler:
    def __init__(self, A, k, Q,N):
        """ Create largest k-leverage scores Sampler for the matrix :math:`A` for k-low rank apparoximation.
        :param A: 
            Matrix :math:`A`.
        :type A: 
            array_type
        :param Q: 
            Matrix containig the right singular vectors of :math:`A`.
        :type Q: 
            array_type
        :param k: 
            The order of low rank apparoximation.
        :type k: 
            int
        :param N: 
            The dimension of subsampling (the number of columns) of A.
        :type N: 
            int
        """
        self.A = A
        # print("k is", type(k))
        self.Q = np.transpose(Q[0:k,:])
        self.N = N
        self.k = k
        
        self.sampling_list = []
        self.lvs_array = self.Estimate_Leverage_Scores()
    def Estimate_Leverage_Scores(self):
        return 1/(self.k)*np.diag(np.dot(self.Q,self.Q.T))
    def MultiRounds(self):
        temp_list = list(reversed(np.argsort(self.lvs_array)))
        sampled_indices_ = temp_list[0:self.k]
        self.sampling_list = sampled_indices_
        return sampled_indices_

def Extract_LLS(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[int(row['uid'])]][int(row['mid'])] = int(row['ratings'])
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)
    A = userItemMatrix.T
    number_of_coreusers = int(u_len * percent_core_user)
    print(number_of_coreusers)
    u4, s4, Q = svds(A, k=number_of_coreusers)
    lls = largest_leveragescores_Sampler(A, number_of_coreusers, Q, number_of_coreusers)
    indx = lls.MultiRounds()
    # indx = indx.
    print("LENGTH OF INDEX:", len(indx))
    print("INDICES ARE:", indx)
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(indx))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

- cur decomposition from paper (https://www.pnas.org/doi/10.1073/pnas.0803205106) called optimal cur

In [17]:
class CUR():
    def __init__(self, k, eps, it=None, truncated=False):
        self.k = k
        print("K IS :", self.k)
        self.eps = eps
        self.trunc = truncated 
        self.c = k * np.log(k) / eps**2 #expectation number of sampled columns
        self.C, self.U, self.R = None, None, None #matrices of decomposition
        self.pi_col, self.pi_row = None, None #leverage scores of corresponding columns/rows
        self.col_indices = None
        self.row_indices = None
    
    def column_select(self, A):
        n = A.shape[1]
        A = np.array(A.copy())
        if self.trunc:
            _, _, v_k = randomized_svd(A, self.k) #for very big matrices
        else:
            _, _, vh = np.linalg.svd(A, full_matrices=False)
            v_k = vh[0:self.k, :]
        
        pi = 1 / self.k * np.sum(v_k**2, axis=0)
        c_index = [np.random.choice(2, 
                        p=[1 - min(1, self.c * pi[i]), min(1, self.c * pi[i])]) for i in range(n)
                  ]
        c_index = np.nonzero(c_index)[0]
        print(len(c_index))
        C = A[:, c_index]
        return C, c_index, pi

    def run_CUR(self, A):
        A = np.array(A.copy())
        # self.C, self.col_indices, self.pi_col = self.column_select(A)
        self.R, self.row_indices, self.pi_row = self.column_select(A.T)
        # self.U = np.linalg.pinv(self.C) @ A @ np.linalg.pinv(self.R.T)
        return self.row_indices
def OPTIMAL_CUR_ExtractCoreUsers(dataframe, unique_user_len, unique_item_len):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)

    for index, row in dataframe.iterrows():
        userItemMatrix[row['uid']][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    mat = userItemMatrix
    print("MAT:", mat)
    print(mat.shape)
    cur = CUR(k=int(u_len * 0.10), eps=0.5,)
    # ids = np.argsort(cur.pi_col)[::-1][:5]
    cur.run_CUR(mat)
    cur_coreusers_idx = np.argsort(cur.pi_row)[::-1][:int(u_len * 0.10)]
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(cur_coreusers_idx))]
    # print(len(cur_coreusers))
    return cur_coreusers

In [18]:
# core_users = CUR_ExtractCoreUsers(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids, percent_core_user=0.40)
# sdf.to_csv('file_name.csv', index=False)
# core_users = ExtractCoreUsers(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids)
# core_users = RandomSampling(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids, percent_core_user=0.40)
core_users = Extract_LLS(core_user_ko_input_train_df, unique_uids, unique_mids, percent_core_user=0.40)
support_user_list = np.unique(core_users['uid'])
print("CORE USERS:" ,core_users)
print("NUMBER OF CORE USERS:", len(support_user_list))

USER LEN: 6040
MOVIE LEN: 3706
USER ITEM MATRIX: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
2416
LENGTH OF INDEX: 2416
INDICES ARE: [4168, 1469, 1679, 4276, 888, 1068, 1149, 1940, 1180, 4226, 3840, 3617, 3031, 1241, 3223, 3538, 1448, 2062, 1014, 1979, 5110, 1265, 548, 1447, 751, 5794, 3390, 1604, 194, 4063, 423, 1087, 4646, 351, 3807, 3828, 2908, 523, 4507, 1697, 849, 1732, 4343, 3525, 6035, 880, 3475, 4681, 3820, 1284, 3823, 677, 5830, 5811, 3777, 2819, 2105, 3609, 4509, 6015, 5642, 5332, 1009, 2985, 2664, 5393, 3984, 2115, 2961, 1018, 1424, 3291, 730, 5953, 4446, 1896, 3791, 3066, 4542, 5099, 2506, 1911, 854, 4978, 698, 5787, 868, 4481, 4424, 2180, 5366, 3883, 2123, 4053, 3506, 549, 3310, 4770, 1163, 1050, 215, 5915, 2776, 672, 3933, 2014, 1836, 3841, 2856, 1670, 4386, 5219, 2108, 1273, 5635, 3704, 2906, 4040, 1446, 4311, 2303, 5073, 1146, 5386, 3490, 1675, 1740, 1957, 1

### CODE SNIPPET FOR GETTING NESTED CORE USERS:

In [19]:
core_users_nested = CUR_ExtractCoreUsers(core_users, len(np.unique(core_users['uid'])), unique_mids, percent_core_user=0.50)
print(len(np.unique(core_users_nested['uid'])))
print("NESTED CORE USERS:\n" ,core_users_nested)

USER LEN: 2416
MOVIE LEN: 3706
USER ITEM MATRIX: 
 [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
MAT: [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(2416, 3706)
[0.00025458 0.00017872 0.0005156  ... 0.00114177 0.00025973 0.00043845]
1208
NESTED CORE USERS:
          uid   mid  ratings
254        4  2775        1
255        4  2140        1
256        4  1087        1
257        4    38        1
258        4   279        1
...      ...   ...      ...
999518  6035   548        1
999519  6035  1024        1
999520  6035  1025        1
999521  6035  1027        1
999522  6035  1867        1

[488441 rows x 3 columns]


In [28]:
core_users_index_list = core_users_nested.index.to_list()
non_core_user_index = (df.index.difference(core_users_nested.index))
non_core_user_index = non_core_user_index.tolist()

core_users_df = df.loc[core_users_index_list]
non_core_user_df = df.loc[non_core_user_index]
print("NON CORE USERS:" ,non_core_user_df)
print("CORE USERS:" ,core_users)

NON CORE USERS:           uid   mid  ratings
0           0  1104        1
1           0   639        1
2           0   853        1
3           0  3177        1
4           0  2162        1
...       ...   ...      ...
1000204  6039  1019        1
1000205  6039  1022        1
1000206  6039   548        1
1000207  6039  1024        1
1000208  6039  1025        1

[511768 rows x 3 columns]
CORE USERS:           uid   mid  ratings
254         4  2775        1
255         4  2140        1
256         4  1087        1
257         4    38        1
258         4   279        1
...       ...   ...      ...
1000204  6039  1019        1
1000205  6039  1022        1
1000206  6039   548        1
1000207  6039  1024        1
1000208  6039  1025        1

[777739 rows x 3 columns]


In [29]:
num_users = len(np.unique(non_core_user_df['uid']))
num_items = len(np.unique(non_core_user_df['mid']))
print("NUM USERS:", num_users)
print("NUM ITEMS:", num_items)


NUM USERS: 4832
NUM ITEMS: 3591


In [30]:
# writing to file coreusers and non-core users
# import pickle
# with open("ml-1m_core_users_non_core_users.pkl", "wb") as f:
#     pickle.dump(core_users_df, f)
#     pickle.dump(non_core_user_df, f)
core_users_df.to_csv("ml-1m_core_users.csv", index=False, header=False)
non_core_user_df.to_csv("ml-1m_non_core_users.csv", index=False, header=False)

#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [13]:
core_users_index_list_nested = core_users_nested.index.to_list()
# non_core_user_index = (train_df.index.difference(core_users.index))
# non_core_user_index = non_core_user_index.tolist()

core_users_df_nested = support_train_df.loc[core_users_index_list_nested]
# non_core_user_df = train_df.loc[non_core_user_index]
# print("NON CORE USERS:" ,non_core_user_df)
print("CORE USERS:", core_users_nested)

CORE USERS:           uid   mid  ratings
799         8  2420        5
800         8   627        4
801         8  2426        3
802         8  3129        5
803         8  3130        3
...       ...   ...      ...
1000203  5230  1018        3
1000204  5230  1019        1
1000205  5230  1022        5
1000206  5230   548        5
1000207  5230  1024        4

[254344 rows x 3 columns]


In [22]:
# print("TEST DF CONTAINS TEST FOR CORE AND NON CORE ENTITIES:\n" ,test_df)
# print(core_users['uid'])
unique_uids_in_support_trian = np.unique(np.array(core_users_df['uid']))
unique_uids_in_query_trian = np.unique(query_train_df['uid'])
print(len(unique_uids_in_support_trian))
support_test_df = support_test_df.loc[support_test_df['uid'].isin(unique_uids_in_support_trian)]
print("SUPPORT TEST DF:" ,support_test_df)
query_test_df = query_test_df
print("QUERY TEST DF:\n", query_test_df)

4847
SUPPORT TEST DF:            uid   mid  ratings
55210        0    12        1
55686       19   478        1
55734       21   413        1
56328       46  1006        1
56445       50  1097        1
...        ...   ...      ...
1498974  55115  6585        1
1499087  55120  1817        1
1499448  55136  7316        1
1499646  55143   633        1
1500199  55163  2131        1

[4847 rows x 3 columns]
QUERY TEST DF:
            uid   mid  ratings
55229        1    34        1
55416        9   219        1
55435       10   237        1
55903       29   682        1
55970       32   740        1
...        ...   ...      ...
1499328  55131   856        1
1499623  55142  5358        1
1500406  55170  2066        1
1500502  55174  4130        1
1500727  55183  4905        1

[6714 rows x 3 columns]


#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [14]:
unique_uids_in_support_trian = np.unique(np.array(core_users_df_nested['uid']))
unique_uids_in_query_trian = np.unique(query_train_df['uid'])
print(len(unique_uids_in_support_trian))
support_test_df = support_test_df.loc[support_test_df['uid'].isin(unique_uids_in_support_trian)]
print("SUPPORT TEST DF:" ,support_test_df)
query_test_df = query_test_df
print("QUERY TEST DF:\n", query_test_df)

523
SUPPORT TEST DF:           uid   mid  ratings
1199        9  1868        5
5581       38  2203        5
7108       47  1155        4
8194       55  1023        3
8695       57  1155        4
...       ...   ...      ...
993008   5997   548        4
994110   6001  1025        4
996791   6015  3540        4
997889   6025  1025        5
1000208  6039  1025        4

[523 rows x 3 columns]
QUERY TEST DF:
          uid   mid  ratings
253        3  1148        5
1359      11  1155        3
1492      13  2765        3
2523      19  1148        4
2545      20  3531        1
...      ...   ...      ...
991938  5992   970        5
993035  5998   527        5
995595  6011  2561        2
998354  6033   527        5
999744  6037  1007        5

[809 rows x 3 columns]


In [23]:
support_train = []
for index,row in core_users_df.iterrows():
    support_train.append([row['uid'], row['mid'], row['ratings']])
query_train = []
for index, row in query_train_df.iterrows():
    query_train.append([row['uid'], row['mid'], row['ratings']])
support_test = []
for index, row in support_test_df.iterrows():
    support_test.append([row['uid'], row['mid'], row['ratings']])
query_test = []
for index, row in query_test_df.iterrows():
    query_test.append([row['uid'], row['mid'], row['ratings']])
user_his_dic = {}
for u in train_ui_dic.keys():
    user_his_dic[u] = train_ui_dic[u]
user_supp_list = np.unique(core_users_df['uid']).tolist()

#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [15]:
support_train = []
for index,row in core_users_df_nested.iterrows():
    support_train.append([row['uid'], row['mid'], row['ratings']])
query_train = []
for index, row in query_train_df.iterrows():
    query_train.append([row['uid'], row['mid'], row['ratings']])
support_test = []
for index, row in support_test_df.iterrows():
    support_test.append([row['uid'], row['mid'], row['ratings']])
query_test = []
for index, row in query_test_df.iterrows():
    query_test.append([row['uid'], row['mid'], row['ratings']])
user_his_dic = {}
for u in train_ui_dic.keys():
    user_his_dic[u] = train_ui_dic[u]
user_supp_list = np.unique(core_users_df_nested['uid']).tolist()

In [17]:
import pickle
with open("COS_ml1m_cur_20_support_as_core.pkl", "wb") as f:
    pickle.dump(support_train, f)
    pickle.dump(query_train, f)
    pickle.dump(support_test, f)
    pickle.dump(query_test, f)
    pickle.dump(user_supp_list, f)
    pickle.dump(user_his_dic, f)